In [1]:
import pandas_gbq
import pandas as pd
from sklearn.metrics import auc, confusion_matrix, precision_recall_curve, roc_auc_score
import glob

The following functions to calculate the model performance can be found at https://github.com/deliveryhero/datahub-airflow/blob/main/dags/mkt/mkt_reorder_performance_pipeline.py

In [2]:
all_terms = [
    "PY_AR", "PY_BO", "PY_CL", "PY_DO", "PY_EC", "PY_GT", 
    "PY_HN", "AP_PA", "PY_PE", "PY_PY", "PY_SV", "PY_UY", "PY_VE", 
    "PY_NI", "HS_SA", "IN_AE", "IN_OM", "IN_QA", "IN_BH", "IN_EG", 
    "TB_AE", "TB_BH", "HF_EG", "TB_IQ", "TB_JO", "FP_HK", "FP_BD", "FP_TH", "FP_TW", "FP_SG", "FP_PH", "FP_PK", 
    "FP_MY", "FP_LA", "FP_MM", "FP_KH", "YS_TR", "PY_GT", "DJ_CZ", "FP_KH", "HS_SA", "OP_SE", "PY_CL", "PY_PE", "PY_EC", "TB_OM", "PY_CR", 
    "PY_SV", "PY_DO", "IN_QA", "PY_HN", "FP_BD", "PY_UY", "IN_EG", "FP_TH", "FP_PH", "FO_NO", 
    "FP_LA", "IN_LB", "FP_MM", "FP_SG", "PO_FI", "PY_BO", "MJM_AT", "TB_IQ", "IN_AE", "FP_PK", 
    "IN_BH", "IN_OM", "NP_HU", "TB_BH", "EF_GR", "TB_KW", "AP_PA", "HF_EG", "TB_QA", "PY_AR", 
    "TB_AE", "FP_HK", "FP_MY", "TB_JO", "FY_CY", "FP_TW", "CG_QA", "YS_TR", "PY_PY", "PY_CR"
]
entities = list(set(all_terms))


In [3]:
def model_evaluation_metrices(y_true, y_pred_binary, ypred_score):
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()
    # auc = roc_auc_score(y_true,y_pred)
    recall = tp / (tp + fn)
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tn + fp + fn + tp)
    f1_score = (2 * tp) / (2 * tp + fp + fn)
    # calculate precision-recall curve
    precision_for_auc, recall_for_auc, thresholds_vals = precision_recall_curve(
        y_true, y_pred_binary
    )
    # calculate precision-recall AUC
    precision_recall_auc = auc(recall_for_auc, precision_for_auc)
    roc_auc = roc_auc_score(y_true=y_true, y_score=ypred_score)

    return (
        round(accuracy, 2),
        round(recall, 2),
        round(specificity, 2),
        round(f1_score, 2),
        round(precision, 2),
        round(roc_auc, 2),
        round(precision_recall_auc, 2),
    )

def make_results(df, filter_col):
    df_store_final = pd.DataFrame(
            columns=[
                "threshold",
                "global_entity_id",
                "lifecycle_segment",
                "accuracy",
                "recall",
                "specificity",
                "f1_score",
                "precision",
                "roc_auc",
                "precision_recall_auc",
            ]
        )

    for segment in df[filter_col].unique():
        df_filtered_ = df[df[filter_col]==segment]
        for geid in entities:
            df_filtered = df_filtered_[df_filtered_["global_entity_id"]==geid]
            for mythres in [0.3, 0.5, 0.7]:
                binary_pred = df_filtered["reorder_score_scaled"].apply(
                    lambda x: 1 if x > mythres else 0
                )
                if (df_filtered["reordered"].nunique() < 2) | (
                    binary_pred.nunique() < 2
                ):
                    continue
    
                (
                    accuracy,
                    recall,
                    specificity,
                    f1_score,
                    precision,
                    roc_auc,
                    precision_recall_auc,
                ) = model_evaluation_metrices(
                    y_true=df_filtered["reordered"].to_list(),
                    y_pred_binary=binary_pred,
                    ypred_score=df_filtered["reorder_score_scaled"],
            )
    
                df_store = pd.DataFrame(
                                    index=[0],
                                    columns=[
                                        "threshold",
                                        "lifecycle_segment",
                                        "global_entity_id",
                                        "accuracy",
                                        "recall",
                                        "specificity",
                                        "f1_score",
                                        "precision",
                                        "roc_auc",
                                        "precision_recall_auc",
                                    ],
                                )
                df_store["threshold"] = mythres
                df_store["lifecycle_segment"] = segment
                df_store["global_entity_id"] = geid
                df_store["accuracy"] = accuracy
                df_store["recall"] = recall
                df_store["specificity"] = specificity
                df_store["f1_score"] = f1_score
                df_store["precision"] = precision
                df_store["roc_auc"] = roc_auc
                df_store["precision_recall_auc"] = precision_recall_auc
                df_store_final = pd.concat(
                    [df_store_final, df_store], axis=0, ignore_index=True
                )
    return df_store_final

In [4]:
q=f'''
WITH
  cust_orders AS (
  SELECT
    global_entity_id,
    analytical_customer_id,
    placed_at_local,
    1 AS reordered,
    CASE
      WHEN is_discount OR is_voucher THEN 1
    ELSE
    0
  END
    AS is_incentivized_reorder,
    CASE
      WHEN is_discount OR is_voucher THEN 0
    ELSE
    1
  END
    AS is_organic_reorder,
    CONCAT(EXTRACT(MONTH
      FROM
        placed_at_local),"-",EXTRACT(YEAR
      FROM
        placed_at_local)) AS month_year,
  FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders`
  WHERE
    partition_date_local BETWEEN DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH)
    AND DATE_ADD(DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH), INTERVAL 27 DAY)
    AND is_successful
    AND analytical_customer_id IS NOT NULL
    AND global_entity_id IN UNNEST({entities}) ),
  cust_first_order_each_month AS (
  SELECT
    * EXCEPT(placed_at_local),
    ROW_NUMBER() OVER (PARTITION BY analytical_customer_id, month_year ORDER BY placed_at_local) AS row_num,
  FROM
    cust_orders ),
  recency AS (
  SELECT
    global_entity_id,
    analytical_customer_id,
    DATE_DIFF(DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH), MIN(placed_at_local), DAY) AS first_order_recency,
    COUNT(DISTINCT order_id) AS orders
  FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders`
  WHERE
    is_successful IS TRUE
    AND analytical_customer_id IS NOT NULL
    AND global_entity_id IN UNNEST({entities})
    AND DATE(partition_date_local) <= DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH)
  GROUP BY
    global_entity_id,
    analytical_customer_id ),
  pred_scores_general AS (
  SELECT
    "general_reorder" AS model_type,
    global_entity_id,
    analytical_customer_id,
    concated_reorder_scores[ORDINAL(28)] AS reorder_score,
    scoring_date,
    CONCAT(EXTRACT(MONTH
      FROM
        scoring_date),"-",EXTRACT(YEAR
      FROM
        scoring_date)) AS month_year
  FROM
    `mkt-reorder-prod.mkt_reorder_prod.historical_predictions_reorder-general-all-ALL`
  WHERE
    scoring_date IN (DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH)) ),
  pred_scores_no_segments AS (
  SELECT
    *
  FROM
    pred_scores_general ),
  pred_scores AS (
  SELECT
    p.*,
    h.orders,
    h.first_order_recency
  FROM
    pred_scores_no_segments p
  LEFT JOIN (
    SELECT
      analytical_customer_id,
      global_entity_id,
      orders,
      first_order_recency
    FROM
      recency 
     WHERE orders in (1,2,3,4) and first_order_recency in (1,2,3,4)
      ) h
  ON
    h.analytical_customer_id = p.analytical_customer_id
    AND h.global_entity_id = p.global_entity_id ),
  merged_table AS (
  SELECT
    p.global_entity_id,
    p.analytical_customer_id,
    COALESCE(c.reordered, 0) AS reordered,
    p.month_year,
    p.reorder_score*100 AS reorder_score,
    p.model_type,
    p.orders,
    p.first_order_recency
  FROM
    pred_scores AS p
  LEFT JOIN (
    SELECT
      * EXCEPT(row_num)
    FROM
      cust_first_order_each_month
    WHERE
      row_num = 1 ) AS c
  ON
    p.global_entity_id = c.global_entity_id
    AND p.analytical_customer_id = c.analytical_customer_id
    AND p.month_year = c.month_year ),
  binned_table AS (
  SELECT
    *,
    CASE
      WHEN reorder_score < 10 THEN "[0,10)"
      WHEN reorder_score >= 10
    AND reorder_score < 20 THEN "[10,20)"
      WHEN reorder_score >= 20 AND reorder_score < 30 THEN "[20,30)"
      WHEN reorder_score >= 30
    AND reorder_score < 40 THEN "[30,40)"
      WHEN reorder_score >= 40 AND reorder_score < 50 THEN "[40,50)"
      WHEN reorder_score >= 50
    AND reorder_score < 60 THEN "[50,60)"
      WHEN reorder_score >= 60 AND reorder_score < 70 THEN "[60,70)"
      WHEN reorder_score >= 70
    AND reorder_score < 80 THEN "[70,80)"
      WHEN reorder_score >= 80 AND reorder_score < 90 THEN "[80,90)"
      WHEN reorder_score >= 90 THEN "[90,100)"
    ELSE
    "invalid_value"
  END
    AS reorder_bin,
  FROM
    merged_table ),
  min_max_reorder AS (
  SELECT
    global_entity_id,
    model_type,
    month_year,
    MAX(reorder_score) AS max_reorder_score,
    MIN(reorder_score) AS min_reorder_score
  FROM
    binned_table
  GROUP BY
    model_type,
    global_entity_id,
    month_year )
SELECT
  r.global_entity_id,
  r.analytical_customer_id,
  r.reordered,
  r.month_year,
  r.reorder_score,
  r.model_type,
  r.reorder_bin,
  (r.reorder_score - m.min_reorder_score)/(m.max_reorder_score - m.min_reorder_score) AS reorder_score_scaled,
  r.orders,
  r.first_order_recency
FROM
  binned_table AS r
JOIN
  min_max_reorder AS m
ON
  m.model_type = r.model_type
  AND m.global_entity_id = r.global_entity_id
  AND m.month_year = r.month_year
'''
print(q)
df = pandas_gbq.read_gbq(q)
print(df.head)


    


WITH
  cust_orders AS (
  SELECT
    global_entity_id,
    analytical_customer_id,
    placed_at_local,
    1 AS reordered,
    CASE
      WHEN is_discount OR is_voucher THEN 1
    ELSE
    0
  END
    AS is_incentivized_reorder,
    CASE
      WHEN is_discount OR is_voucher THEN 0
    ELSE
    1
  END
    AS is_organic_reorder,
    CONCAT(EXTRACT(MONTH
      FROM
        placed_at_local),"-",EXTRACT(YEAR
      FROM
        placed_at_local)) AS month_year,
  FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders`
  WHERE
    partition_date_local BETWEEN DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH)
    AND DATE_ADD(DATE_TRUNC(DATE_SUB('2024-01-01', INTERVAL 1 MONTH), MONTH), INTERVAL 27 DAY)
    AND is_successful
    AND analytical_customer_id IS NOT NULL
    AND global_entity_id IN UNNEST(['PY_UY', 'PO_FI', 'PY_BO', 'PY_DO', 'DJ_CZ', 'IN_OM', 'FP_SG', 'FP_MY', 'PY_AR', 'MJM_AT', 'FY_CY', 'NP_HU', 'FP_MM', 'EF_GR', 'IN_BH', 'TB_QA', 'FO_NO', 'F

/Users/a.capuano/opt/miniconda3/envs/ml/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)

KeyboardInterrupt



In [ ]:
results = make_results(df, filter_col='orders')
results_rec = make_results(df, filter_col='first_order_recency')
    


In [ ]:
results.rename(columns={"lifecycle_segment": "orders"}, inplace=True)
results_rec.rename(columns={"lifecycle_segment": "orders"}, inplace=True)

In [ ]:
melted_results = pd.melt(results, id_vars=['global_entity_id', 'orders', 'threshold'], value_vars=['accuracy', 'f1_score', 'precision_recall_auc'], var_name='performance_metric', value_name='value')
melted_results=melted_results.sort_values(['global_entity_id', 'threshold', 'orders', 'performance_metric'])
melted_results.to_csv('all_2024_01_01_performance_orders_number.csv')


In [ ]:
melted_results_rec = pd.melt(results_rec, id_vars=['global_entity_id', 'orders', 'threshold'], value_vars=['accuracy', 'f1_score', 'precision_recall_auc'], var_name='performance_metric', value_name='value')
melted_results_rec=melted_results_rec.sort_values(['global_entity_id', 'threshold', 'orders', 'performance_metric'])
melted_results_rec.to_csv('all_2024_01_01_performance_first_order_recency.csv')
